# Описание проекта

## Задача

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Описание данных

**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.\
**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

# Выполнение проекта

## Загрузка данных

>Загрузите и изучите данные.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
df =  pd.read_csv("https://code.s3.yandex.net/datasets/insurance.csv")

In [3]:
df.shape

(5000, 5)

In [4]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

>Ответьте на вопрос и обоснуйте решение.
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.):
- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** b) При умножении на обратимую матрицу качество линейной регрессии не изменится.

**Обоснование:** 

Надо доказать, что $a1 = a$, 

$$a1 = XPw$$

$$a1 = XP((XP)^T (XP))^{-1})(XP^T)y$$

Доказательство сводится к доказательству равенства:
$$X(X^T X)^{-1} X^T y = XP((XP)^T (XP))^{-1})(XP^T)y$$

Доказательство:
$$XP((XP)^T (XP))^{-1} (XP)^T y$$
$$XP((P^T X^T) (XP))^{-1} (P^T X^T) y$$
$$XP (XP)^{-1} (P^T X^T)^{-1}(P^T X^T) y$$
$$XP P^{-1} X^{-1} (X^T)^{-1} (P^T)^{-1}   (P^T X^T) y$$
$P^{-1}, P, и (P^T)^{-1}, P^T$ сокращаются и остается выражение:
$$X X^{-1} (X^T)^{-1}(X^T) y$$
$$X(X^T X)^{-1} X^T y$$

Что и требовалось доказать.

## Алгоритм преобразования

>Предложите алгоритм преобразования данных для решения задачи. Обоснуйте, почему качество линейной регрессии не поменяется.

**Алгоритм**
1. Сгенерировать случайную квардратную матрицу размерности n - 1 х n - 1, где n - количество признаков в исходном наборе данных, предварительно задав условия для генератора случайных чисел.
2. Сохранить сгенерированную матрицу в переменную.
3. Проверить матрицу на обратимость.
4. Если матрица необратима, повторить п1-п3.
5. Если матрица обратима, умножить сгенерированную матрицу на матрицу признаков.

**Обоснование**

Качество линейной регресси не поменяется в связи с доказательством из п 2.2.

In [7]:
n = df.shape[1] - 1

In [8]:
features = df.drop("Страховые выплаты", axis = 1).values

In [9]:
features

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [10]:
np.random.seed(47)
matrix = np.random.rand(n,n)

In [11]:
np.linalg.inv(matrix)

array([[-0.77365149,  0.78506938,  0.78534112, -0.22480192],
       [ 1.50852384,  0.1197712 , -0.11204187, -1.79127915],
       [ 0.97115158, -2.76306708,  2.28794006,  2.32552138],
       [-3.10111545,  5.14337675, -4.68676045,  0.21736189]])

In [12]:
target = np.array(df["Страховые выплаты"])

In [13]:
target

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [14]:
features_modif = features @ matrix

In [15]:
features_modif

array([[35048.36218424, 12267.86804422, 12725.06221873,  1208.63542526],
       [26861.82699069,  9409.75875555,  9758.04909031,   931.8252474 ],
       [14847.17178574,  5202.82555072,  5394.56235104,   516.26182566],
       ...,
       [23948.80083683,  8378.00700706,  8692.33636945,   822.92131357],
       [23103.19104801,  8084.90187767,  8387.80616591,   795.61056356],
       [28684.88421598, 10037.62874843, 10412.73803293,   987.14344306]])

## Проверка алгоритма

>Запрограммируйте этот алгоритм, применив матричные операции. Проверьте, что качество линейной регрессии из sklearn не отличается до и после преобразования. Примените метрику R2.

In [16]:
X_train = features[:4000] 
X_test = features[4000:]
X_train_modif = features_modif[:4000]
X_test_modif = features_modif[4000:]
y_train = target[:4000]
y_test = target[4000:]

In [17]:
model = LinearRegression()

In [18]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)

In [19]:
r2

0.4373576608459112

In [20]:
model.fit(X_train_modif, y_train)
y_pred_modif = model.predict(X_test_modif)
r2_modif = r2_score(y_test, y_pred_modif)

In [21]:
r2_modif

0.4373576608463339

In [22]:
round(r2_modif - r2, 10)

0.0

# Вывод

- Умножение матрицы признаков на обратимую квадратную матрицу не ухудшает качество линейной рергессии. 